<h1><center>Choropleth Zipcode Plot on Mapbox</center></h1>

In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())

<h4>Read in Earthquake Event Zipcode Data</h4>

In [ ]:
cdi_df = pd.read_csv('./data/test.csv')

In [ ]:
# geo_df = gpd.GeoDataFrame(cdi_df, geometry=gpd.points_from_xy(cdi_df.Longitude, cdi_df.Latitude))
cdi_df.rename({'# Columns: ZIP/Location': 'ZIP/Location'}, axis=1, inplace=True)

<h4> Read in South Carolina Zipcodes Shapefile</h4>

In [ ]:
state_zip = gpd.read_file(r"./data/zipcode_data/tl_2010_45_zcta510.shp")

In [ ]:
state_zip[state_zip['ZCTA5CE10'] == '29229']

#### Make The Merge Column of the Two Files The Same Type

In [ ]:
cdi_df['ZIP/Location'] = cdi_df[['ZIP/Location']].astype('str')

In [ ]:
state_zip['ZCTA5CE10'] = state_zip[['ZCTA5CE10']].astype('str')

#### Merge The CDI Zipcode Dataframe With SC State Zipcode Shapefile Dataframe

In [ ]:
df = cdi_df
geo_dff = gpd.GeoDataFrame(state_zip).merge(df, left_on="ZCTA5CE10", right_on='ZIP/Location').set_index('ZIP/Location')

#### Plot Zipcode Choropleth Mapbox Map
Each colored area is a zipcode where there were DYFI responses.

In [ ]:
fig = px.choropleth_mapbox(geo_dff,
                           geojson=geo_dff.geometry,
                           locations=geo_dff.index,
                           color="CDI",
                           range_color=[0.0, 10.0],
                           center={"lat": 34.0007, "lon": -81.0348},
                           mapbox_style="streets",
                           # mapbox_style="basic",
                           opacity=0.35,
                           zoom=7.5,
                           hover_data={'ZCTA5CE10': False, 'No. of responses': True},)
                           # labels={'ZCTA5CE10': 'ZIP Code'})

# Plot the epicenter
fig.add_trace(go.Scattermapbox(lon=[-80.794252],
                               lat=[34.170983],
                               showlegend=False,
                               mode='markers+lines',
                               marker={'size': 12, 'opacity': 1, 'symbol': ['star']},
                               hovertemplate='%{lon}, %{lat}<extra></extra>',))

fig.update_layout(autosize=True, margin=dict(t=0, b=0, l=0, r=0), showlegend=True)
fig.show()
